In [1]:
import torchlensmaker as tlm
import torch

import json

from torchlensmaker.new_light_sources.light_sources_elements import GenericLightSource
from torchlensmaker.new_light_sources.source_geometry_elements import ObjectGeometry2D

from torchlensmaker.new_sampling.sampling_elements import LinspaceSampling1D
from torchlensmaker.new_material.material_elements import NonDispersiveMaterial


def describe(name, t):
    print(name)
    print(t.shape)
    print(t.min(), t.max())
    print()

light_source = GenericLightSource(
        pupil_sampler = LinspaceSampling1D(6),
        field_sampler= LinspaceSampling1D(6),
        wavel_sampler= LinspaceSampling1D(6),
        material = NonDispersiveMaterial(1.5),
        geometry = ObjectGeometry2D(beam_angular_size=20, object_diameter=5, wavelength=(600, 800)),
)

# high level interface with mixed dim, string material
# l2 = ObjectAtInfinity(beam_angular_size=20, object_diameter=5, wavelength=(600, 800), material="water")

optics = tlm.Sequential(
    light_source,
    tlm.Gap(10),
    tlm.RefractiveSurface(tlm.Sphere(15, 50), material="BK7"),
    tlm.Gap(50),
    tlm.Aperture(50)
)

print(optics)

data = tlm.default_input(dim=2, dtype=torch.float64, sampling={"base": 5, "object": 3, "wavelength": 3})
outputs = optics(data)

describe("P", outputs.P)

describe("base", outputs.rays_base)
describe("object", outputs.rays_object)
describe("wavel", outputs.rays_wavelength)

scene = tlm.show2d(optics, return_scene=True)
#json.dump(scene, open("testnb.json", "w"))


Sequential(
  (0): GenericLightSource(
    (pupil_sampler): LinspaceSampling1D()
    (field_sampler): LinspaceSampling1D()
    (wavel_sampler): LinspaceSampling1D()
    (material): NonDispersiveMaterial()
    (geometry): ObjectGeometry2D()
  )
  (1): Gap(
    (mixed_dim): MixedDim(
      (module_2d): Translate2D()
      (module_3d): Translate3D()
    )
  )
  (2): RefractiveSurface(
    (collision_surface): CollisionSurface()
  )
  (3): Gap(
    (mixed_dim): MixedDim(
      (module_2d): Translate2D()
      (module_3d): Translate3D()
    )
  )
  (4): Aperture(
    (collision_surface): CollisionSurface()
  )
)
P
torch.Size([216, 2])
tensor(-12.9785) tensor(60.)

base
torch.Size([216])
tensor(-10.) tensor(10.)

object
torch.Size([216])
tensor(-2.5000) tensor(2.5000)

wavel
torch.Size([216])
tensor(600.) tensor(800.)

